In [1]:
import threading
import cv2
import mediaPipeHandler as mph
import time
import gestures

In [2]:
# Zoom-In and Zoom-Out
zoom_factor = 1.0  # Initial zoom level
zoom_increment = 0.1  # Zoom sensitivity

In [3]:
# Printing Commands on the screen 
def print_command(frame,command_text):
    cv2.putText(frame, command_text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

In [4]:
def gesture_detection(stop_event):
    # Ripple Trail Configuration
    trail = []

    # Define trail properties
    trail_max_length = 10  # Maximum number of points in the trail
    trail_color = (0, 255, 0)  # Green color
    trail_start_radius = 10  # Initial circle radius
    # Open the camera
    camera = cv2.VideoCapture(0)

    # previous_index of x and y are stored for scroll direction check.
    previous_index_x = None
    previous_index_y = None
    previous_distance = None

    last_gesture_time = time.time()

    while not stop_event.is_set():
        isCapturedFrameSuccessful, frame = camera.read()
        if not isCapturedFrameSuccessful:
            break

        frame = cv2.flip(frame, 1)
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process hand landmarks
        hand_process_result = mph.hands.process(framergb)

        if hand_process_result.multi_hand_landmarks:
            for hand_landmarks in hand_process_result.multi_hand_landmarks:
                landmarks = []
                for idx, lm in enumerate(hand_landmarks.landmark):  # Iterate through each landmark
                    x = int(lm.x * frame.shape[1])  # Get x coordinate
                    y = int(lm.y * frame.shape[0])  # Get y coordinate
                    landmarks.append([x, y])

                # Draw connections first
                mph.mpDraw.draw_landmarks(frame, hand_landmarks, mph.mpHands.HAND_CONNECTIONS, mph.default_landmark_spec, mph.default_connection_spec)

                # Draw the filled green circle for the index finger tip (after drawing connections)
                index_tip = landmarks[8]  # Index finger tip

                cv2.circle(frame, (index_tip[0], index_tip[1]), 5, (0, 255, 0), -1)  # Filled green circle

                # Add the index finger position to the trail
                trail.append(index_tip)

                # Limit the trail length
                if len(trail) > trail_max_length:
                    trail.pop(0)

                # Draw the ripple trail
                for i, point in enumerate(trail):
                    # Calculate the radius and opacity for each circle
                    radius = trail_start_radius - int((i / trail_max_length) * trail_start_radius)
                    opacity = int(255 * (1 - (i / trail_max_length)))
                    color = (trail_color[0], trail_color[1], trail_color[2], opacity)  # RGBA

                    # Draw the fading circle
                    cv2.circle(frame, point, radius, trail_color, 1)

                # Detect gestures
                if gestures.is_click_gesture(landmarks) and time.time() - last_gesture_time > 1:
                    print_command(frame, "Click")
                    last_gesture_time = time.time()

                elif gestures.is_scroll_gesture(landmarks) and time.time() - last_gesture_time > 1:
                    direction, previous_index_x, previous_index_y = gestures.detect_scroll_direction(landmarks, previous_index_x, previous_index_y)
                    if direction != "none":
                        print_command(frame, "Scrolling: " + direction)
                        last_gesture_time = time.time()

                elif gestures.is_zoom_detected(landmarks) and time.time() - last_gesture_time > 1:
                    zoom_direction, previous_distance = gestures.detect_zoom_direction(landmarks, previous_distance)
                    if zoom_direction != "":
                        print_command(frame, "Zooming: " + zoom_direction)
                        last_gesture_time = time.time()

                

        # Display the frame
        cv2.imshow("Output", frame)
        if cv2.waitKey(1) == ord('q') or stop_event.is_set():
            stop_event.set()
            break

    camera.release()
    cv2.destroyAllWindows()


In [5]:
stop_event = threading.Event()

# Start the gesture detection in a separate thread
gesture_thread = threading.Thread(target=gesture_detection, args=(stop_event,), daemon=True)
gesture_thread.start()

# Main UI code (Example loop)
try:
    while not stop_event.is_set():
        time.sleep(1)

        # Check for quit key press to stop both threads
        if cv2.waitKey(1) == ord('q'):
            stop_event.set()
            break
finally:
    # Ensure the gesture thread is properly terminated
    gesture_thread.join()
    print("Application exited gracefully.")

Application exited gracefully.
